In [1]:
import warnings
warnings.filterwarnings("ignore")

import re
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch

from DataLoader.DataLoader import DataLoader
from DataLoader.DataBasedAgent import DataBasedAgent
from DataLoader.DataRLAgent import DataRLAgent
import DeepRLAgent.VanillaInput.Train as Train
from PatternDetectionInCandleStick.Evaluation import Evaluation
import distinctipy
import talib

from importlib import reload

Train = reload(Train)
DeepRL = Train.Train
from utils_best_arm import add_train_portfo, add_test_portfo, plot_return, calc_return, plot_action_point, setup_logger
pd.options.display.max_colwidth = 100
from scipy.optimize import minimize

device = "cpu"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import talib
CURRENT_PATH = os.getcwd()

In [2]:
def train(
    DATASET_NAME, 
    split_point='2018-01-01', 
    begin_date='2010-01-01', 
    end_date='2020-08-24', 
    model_start_date="",
    model_end_date="",
    initial_investment=1000,
    transaction_cost=0.0001,
    load_from_file=True,
    reward_type="profit",
    seed=42, 
    state_mode=1,
    n_episodes=5,
    lamb=0.0001,
    GAMMA=0.7, 
    n_step=5, 
    BATCH_SIZE=10, 
    ReplayMemorySize=20,
    TARGET_UPDATE=5,
    window_size=None, 
    train_portfolios={},
    test_portfolios={},
    arms=[],
    show_all = False,
    ratio_threshold=0.9,
):
    data_loader = DataLoader(DATASET_NAME, split_point=split_point, begin_date=begin_date, end_date=end_date, load_from_file=load_from_file)
    
    dataTrain_agent = DataRLAgent(data_loader.data_train, state_mode, 'action_encoder_decoder', device, GAMMA, n_step, BATCH_SIZE, window_size, transaction_cost)
    dataTest_agent = DataRLAgent(data_loader.data_test, state_mode, 'action_encoder_decoder', device, GAMMA, n_step, BATCH_SIZE, window_size, transaction_cost)
    
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

    agent = DeepRL(data_loader, dataTrain_agent, dataTest_agent, 
                DATASET_NAME,  state_mode, window_size, transaction_cost,
                BATCH_SIZE=BATCH_SIZE, GAMMA=GAMMA, ReplayMemorySize=ReplayMemorySize,
                TARGET_UPDATE=TARGET_UPDATE, n_step=n_step, arms=arms)
    
    path = f"./Results/{DATASET_NAME}/{model_start_date}~{model_end_date}/{seed}/train"
    arm = arms[0]
    name = f'{arm["name"]}_{arm["lamb"]}'
    model_path = f"{path}/model_{name}_{seed}.pkl"
    agent_test = agent.test_CCI(initial_investment=initial_investment, test_type='test', model_path=model_path, symbol=DATASET_NAME)
    test_portfolio = agent_test.get_daily_portfolio_value()
    test_portfolio = pd.Series(test_portfolio).pct_change(1).fillna(0).values.tolist() 
    model_name = f'DQN-stock:{DATASET_NAME}-reward:{name}-seed:{seed}'
    return data_loader, {"name": model_name, "portfo": test_portfolio}

In [34]:
initial_investment = 1000


kwargs = {
    "load_from_file": True, 
    "transaction_cost": 0.0000,
    "initial_investment": initial_investment,
    "state_mode": 1,
    "GAMMA": 0.7, 
    "n_step": 5, 
    "BATCH_SIZE": 10, 
    "ReplayMemorySize": 20,
    "TARGET_UPDATE": 5,
    "window_size": None, 
    "lamb": 0.0,
}

_file = "AAPL"

_begin_date = '20{}-01-01'
_end_date = '20{}-01-01'
_split_point = '20{}-01-01' 

arms = [
    { "name": "profit", "lamb": 0},
]

dates = [
    ("2019-06-23", "2020-06-22", "2016-01-01", "2019-01-01"),
    ("2020-06-22", "2021-06-22", "2017-01-01", "2020-01-01"),
    ("2021-06-23", "2022-06-23", "2018-01-01", "2021-01-01"),
]

random_seeds = 1
results = []
portfolios_saved = {}
files = sorted(os.listdir("./Data/"))
for _file in files[:]:
    print(_file)
    portfolios_saved[_file] = {}

    for idx, arm in enumerate(arms[:]):
        results2 = []
        portfolios_saved[_file][f"{arm['name']}-{arm['lamb']}"] = []
        
        for seed in tqdm(range(random_seeds)):
            
            ls = []
            bhs = []
        
            train_portfolios = {}
            test_portfolios = {}
            tmp_result = []
            
            for date in dates:
                model_start_date = date[2]
                model_end_date = date[3]
                split_point = date[0]
                end_date = date[1]

                kwargs.update({
                    "begin_date": "2016-01-01", 
                    "end_date": end_date, 
                    "split_point": split_point,
                    "model_start_date": model_start_date,
                    "model_end_date": model_end_date,
                    "DATASET_NAME": _file,
                    "reward_type": "",
                    "seed": seed,
                    "n_episodes": 140,
                    "arms": [arm],
                    "show_all": True,
                    "ratio_threshold": 3,
                    "train_portfolios": train_portfolios,
                    "test_portfolios": test_portfolios,
                })

                data_loader, model = train(**kwargs)
                ls.extend(model["portfo"])
                bh = data_loader.data_test_with_date["close"]
                bhs.append(bh)
                tmp_result.extend(model["portfo"])

            add_test_portfo(test_portfolios, seed, ls)
            if seed == 0: 
                bhs = pd.concat(bhs, axis=0)
                bh_percentage = bhs.pct_change(1).fillna(0).values
                add_test_portfo(test_portfolios, 'B&H', bh_percentage)
            indexes = calc_return(bh_percentage, test_portfolios)
            results2.append(indexes)
            portfolios_saved[_file][f"{arm['name']}-{arm['lamb']}"].append(tmp_result)

        path = f"./Results/{_file}/exp3_concat"
        if not os.path.exists(path):
            os.mkdir(path)
        
        save_path = f"{path}/CCI.csv"

        portfolios_saved[_file][f"{arm['name']}-{arm['lamb']}"].insert(0, bh_percentage.tolist())
        _df_ = pd.DataFrame(portfolios_saved[_file][f"{arm['name']}-{arm['lamb']}"]).T.fillna(0)
        _df_.to_csv(save_path, index=False)

        results2_df = pd.concat(results2, axis=1)
        results2_bh = results2_df["B&H"]
        del results2_df["B&H"]
        final = pd.concat([
            results2_bh,
            results2_df.median(axis=1)
        ], axis=1)
        final.columns = [f"{_file}-B&H", f"{_file}-{arm['name']}-{arm['lamb']}"]
        if idx > 0:
            del final[f"{_file}-B&H"]
        results.append(final)


AAPL


100%|██████████| 1/1 [00:00<00:00,  1.68it/s]


AMGN


100%|██████████| 1/1 [00:00<00:00,  1.91it/s]


AXP


...

100%|██████████| 1/1 [00:00<00:00,  1.28it/s]


WBA


100%|██████████| 1/1 [00:00<00:00,  1.23it/s]


WMT


100%|██████████| 1/1 [00:00<00:00,  1.14it/s]


In [4]:
def output_bh(symbol):
    raw_df = pd.read_csv(f"./Data/{symbol}/{symbol}.csv")
    raw_df = raw_df[["Date", "Close"]]
    raw_df["pct"] = raw_df["Close"].pct_change(1)
    raw_df = raw_df.query("Date >= '2016-01-01'")
    raw_df = raw_df.set_index("Date")
    del raw_df["Close"]
    raw_df = raw_df.reset_index()
    dates = [
        ("2019-06-23", "2020-06-22", "2016-01-01", "2019-01-01"),
        ("2020-06-22", "2021-06-22", "2017-01-01", "2020-01-01"),
        ("2021-06-23", "2022-06-23", "2018-01-01", "2021-01-01"),
    ]
    ls = []
    for date in dates:
        date1, date2, _, _ = date
        ls.append(raw_df.query(f"Date >= '{date1}' & Date <= '{date2}'"))
    bh = pd.concat(ls)
    bh.reset_index(inplace=True)
    del bh["index"]
    return bh

symbol = "AAPL"
# output_bh(symbol)

In [9]:
symbols = list(portfolios_saved.keys())[:]
# plt.style.use("ggplot")
# plt.rcParams["text.color"] = "black"

ls = []
for symbol in symbols:
    bh = output_bh(symbol)
    res = pd.concat([
        bh,
        pd.DataFrame(portfolios_saved[symbol]["profit-0"]).T
    ], axis=1).dropna()
    res.columns = ["Date", f"{symbol}-B&H", f"{symbol}-CCI"]
    # NOTE cumreturn
    # res[f"{symbol}-B&H"] = (1 + res[f"{symbol}-B&H"]).cumprod() - 1
    # res[f"{symbol}-CCI"] = (1 + res[f"{symbol}-CCI"]).cumprod() - 1 
    date_ls = res["Date"].tolist()
    del res["Date"]
    ls.append(res)


results_cumreturn = pd.concat(ls, axis=1)   
results_cumreturn["date"] = date_ls 
results_cumreturn = results_cumreturn.set_index("date")
results_cumreturn.to_csv("./ts-run-results/[exp3]CCI-returns.csv",)
# results_cumreturn.to_csv("./ts-run-results/[exp3]CCI-cumreturns.csv")
results_cumreturn

,AAPL-B&H,AAPL-CCI,AMGN-B&H,AMGN-CCI,AXP-B&H,AXP-CCI,BA-B&H,BA-CCI,CAT-B&H,CAT-CCI,...,UNH-B&H,UNH-CCI,V-B&H,V-CCI,VZ-B&H,VZ-CCI,WBA-B&H,WBA-CCI,WMT-B&H,WMT-CCI
date,,,,,,,,,,,,,,,,,,,,,
2019-06-24,-0.001006,0.000000e+00,-0.012185,0.000000,-0.004730,0.000000e+00,0.005782,0.000000,0.003585,0.000000,...,-0.010465,0.000000,0.002364,0.000000e+00,0.008655,0.000000e+00,-0.009533,0.0,0.000990,0.000000
2019-06-25,-0.015158,0.000000e+00,-0.002110,0.000000,-0.007894,0.000000e+00,-0.012487,0.000000,-0.004912,0.000000,...,-0.007931,0.000000,-0.014783,0.000000e+00,-0.004634,0.000000e+00,0.019249,0.0,-0.004675,0.000000
2019-06-26,0.021629,0.000000e+00,-0.006886,-0.006886,0.003491,0.000000e+00,0.015217,0.015217,0.010769,0.010769,...,-0.016918,-0.016918,-0.001284,-1.284369e-03,-0.017414,-1.741384e-02,-0.010765,0.0,-0.005058,-0.005058
2019-06-27,-0.000300,0.000000e+00,0.004640,0.004640,0.002832,0.000000e+00,-0.029125,-0.029125,0.002590,0.002590,...,0.011788,0.011788,0.000994,9.938594e-04,0.004562,4.562076e-03,0.040855,0.0,-0.000545,-0.000545
2019-06-28,-0.009112,0.000000e+00,0.001413,0.001413,-0.004034,0.000000e+00,-0.000027,-0.000027,0.005830,0.005830,...,-0.009458,-0.009458,0.013549,1.354889e-02,-0.002096,-2.095940e-03,0.002751,0.0,0.003542,0.003542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-16,-0.039651,-2.220446e-16,-0.020672,0.000000,-0.059636,2.220446e-16,-0.002692,0.054376,-0.054417,-0.000918,...,-0.014243,0.000000,-0.036246,2.220446e-16,-0.006307,-8.671076e-03,-0.026025,0.0,0.010387,0.000000
2022-06-17,0.011533,0.000000e+00,0.017381,0.000000,0.048582,0.000000e+00,0.025795,0.000000,-0.011038,0.000000,...,-0.008836,0.000000,0.005078,0.000000e+00,0.003890,-2.220446e-16,-0.008823,0.0,-0.019317,0.000000
2022-06-21,0.032761,0.000000e+00,0.015721,0.000000,0.004023,0.000000e+00,-0.000366,0.000000,0.019363,0.000000,...,0.062514,0.000000,0.023051,0.000000e+00,0.033041,0.000000e+00,0.019837,0.0,0.032801,0.000000


58

In [32]:
results_df = pd.concat(results, axis=1)
cols = results_df.columns
ls = []
for col in cols:
    if "B&H" in col:
        ls.append(col)
    else:
        symbol = col.split("-")[0]
        ls.append(f"{symbol}-CCI")

results_df.columns = ls
results_df.to_csv(f"./ts-run-results/[exp3]CCI_reward.csv")

def find(symbol):
    ls = []
    for col in results_df.columns:
        if symbol in col:
            ls.append(col)

    return results_df[ls]

files = os.listdir("./Results/")
df = find(files[0])
df


,AAPL-B&H,AAPL-CCI
sortino_test,1.617304,1.817498
sharpe_test,1.170390,1.349268
risk_test,0.350629,0.227505
mdd_test,0.314273,0.199359
downrisk_test,0.253739,0.168894
cumreturn_test,1.853616,1.335226


In [33]:
results_df

,AAPL-B&H,AAPL-CCI,AMGN-B&H,AMGN-CCI,AXP-B&H,AXP-CCI,BA-B&H,BA-CCI,CAT-B&H,CAT-CCI,...,UNH-B&H,UNH-CCI,V-B&H,V-CCI,VZ-B&H,VZ-CCI,WBA-B&H,WBA-CCI,WMT-B&H,WMT-CCI
sortino_test,1.617304,1.817498,0.840163,0.648764,0.495938,-0.387433,-0.372984,-0.095640,0.703495,0.687743,...,1.216106,0.405619,0.413782,-0.328469,0.180898,-0.158368,0.091630,-0.438414,0.445148,1.001242
sharpe_test,1.170390,1.349268,0.571175,0.470785,0.347496,-0.338348,-0.268060,-0.095197,0.522063,0.505123,...,0.908069,0.296560,0.307648,-0.332762,0.130221,-0.163528,0.065749,-0.478067,0.333236,0.711160
risk_test,0.350629,0.227505,0.277822,0.173731,0.440863,0.238369,0.593144,0.298421,0.342866,0.223435,...,0.331632,0.189429,0.325757,0.188447,0.191906,0.108134,0.356151,0.228618,0.242593,0.158377
mdd_test,0.314273,0.199359,0.245339,0.252884,0.496385,0.356927,0.751524,0.463080,0.385543,0.191516,...,0.358999,0.183503,0.363649,0.369693,0.198201,0.173438,0.444402,0.453359,0.257397,0.136064
downrisk_test,0.253739,0.168894,0.188874,0.126070,0.308906,0.208169,0.426286,0.297040,0.254441,0.164105,...,0.247631,0.138497,0.242201,0.190910,0.138145,0.111657,0.255557,0.249296,0.181604,0.112492
cumreturn_test,1.853616,1.335226,0.436128,0.223210,0.189479,-0.280494,-0.635286,-0.196984,0.433928,0.303713,...,1.094994,0.122396,0.152682,-0.216055,0.020123,-0.068645,-0.113478,-0.335962,0.167342,0.353025
